<a href="https://www.linkedin.com/in/amit-maindola-51801423/"><img src = "https://cdn-images-1.medium.com/max/1200/1*u16a0WbJeckSdi6kGD3gVA.jpeg" width = 400> </a>

<h1 align="center"><font size=5 color="DE5538">Clustering Neighborhoods and Restaurants in Chandigarh</font></h1>

## Introduction
In this Notebook I will be clustering the data I had collected in the previous part for Chandigarh. 

### Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3 color="black">

1. <a href="#item1">Cluster Neighborhoods</a>

2. <a href="#item2">Examine Clusters</a>    

3. <a href="#item3">Conclusion</a>   

</font>
</div>

### Import all the required libraries and dependencies

In [1]:
import numpy as np              # library to handle data in a vectorized manner
import pandas as pd             # library for data Analysis
import requests                 # library to handle requests
from bs4 import BeautifulSoup   # library to read the web page

import json                     # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
print("Imported Libraries...")

Imported Libraries...


Let's save the required paths into variables

In [2]:
csv_path=os.path.join(os.getcwd(),'csv/')
raw_path=os.path.join(os.getcwd(),'raw_data/')
img_path=os.path.join(os.getcwd(),'img/')

<a id=item1><font size=4 color="229C75">Cluster Neighborhoods</font></a>

##### Create an Area DataFrame from the CSV file
Split top Cuisines of each area into multiple columns

In [3]:
chd_cluster_areas_df = pd.read_csv(os.path.join(csv_path,"zomato_city_areas.csv"),usecols=['area','popularity_index','nightlife_index'
                                                                                   ,'restaurants_count','top_cuisines'])

# Split the top_cuisines into multiple columns
areas_cuisine = pd.DataFrame(chd_cluster_areas_df['top_cuisines'].str.split(',',expand=True))
areas_cuisine.columns=['cuisine_1','cuisine_2','cuisine_3','cuisine_4','cuisine_5']
# Add the cuisines to the Clustering Dataframe 
chd_cluster_areas_df = chd_cluster_areas_df.join(areas_cuisine,how='outer').drop(columns='top_cuisines')

# View the Areas
chd_cluster_areas_df.head()

,area,popularity_index,nightlife_index,restaurants_count,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5
0,Sector 26,3.77,1.02,6,North Indian,Fast Food,Chinese,Bakery,Street Food
1,Sector 21,4.97,5.00,5,North Indian,Chinese,Fast Food,Continental,Italian
2,Sector 11,4.60,4.16,5,North Indian,Fast Food,Chinese,Continental,Cafe
3,Sector 10,3.77,1.02,5,North Indian,Fast Food,Chinese,Bakery,Street Food
4,Sector 17,4.47,3.32,2,North Indian,Fast Food,Cafe,Chinese,Italian


##### Perform one Hot Encoding for top Cuisines of Area

In [4]:
# Get the onehot encoding for cuisine
def get_cusine_onehot(df,column,drop_col=0):
    onehot = pd.get_dummies(df[[column]],prefix="",prefix_sep="")
    # Drop the column from dataframe
    if drop_col > 0:
        df.drop(columns=[column],inplace=True)
    # Loop through the new columns of encoded values
    for col in onehot.columns.tolist():
        # Check if column already present in the Dataframe
        if col in df.columns.tolist():
            df[col] = df[col] | onehot[col]
        else:
            df[col] = onehot[col]
    return df

In [5]:
get_cusine_onehot(chd_cluster_areas_df,'cuisine_1',drop_col=1)
get_cusine_onehot(chd_cluster_areas_df,'cuisine_2',drop_col=1)
get_cusine_onehot(chd_cluster_areas_df,'cuisine_3',drop_col=1)
get_cusine_onehot(chd_cluster_areas_df,'cuisine_4',drop_col=1)
get_cusine_onehot(chd_cluster_areas_df,'cuisine_5',drop_col=1)

# View the new Values
chd_cluster_areas_df.head()

,area,popularity_index,nightlife_index,restaurants_count,North Indian,Chinese,Fast Food,Cafe,Continental,Bakery,Beverages,Street Food,Italian,Mithai
0,Sector 26,3.77,1.02,6,1,1,1,0,0,1,0,1,0,0
1,Sector 21,4.97,5.00,5,1,1,1,0,1,0,0,0,1,0
2,Sector 11,4.60,4.16,5,1,1,1,1,1,0,0,0,0,0
3,Sector 10,3.77,1.02,5,1,1,1,0,0,1,0,1,0,0
4,Sector 17,4.47,3.32,2,1,1,1,1,0,0,0,0,1,0


##### Cluster each area based on different Parameters

Run *k*-means to cluster the neighborhood into 5 clusters.

In [6]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chd_cluster_areas_df.drop(columns='area'))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Add labels to cluster DataFrame
chd_cluster_areas_df['cluster_label'] = kmeans.labels_

# Take the new DataFrame for Merging
chd_areas_df = pd.read_csv(os.path.join(csv_path,"zomato_city_areas.csv"))

# Update the areas DataFrame
chd_areas_df = pd.merge(chd_areas_df,chd_cluster_areas_df[['area','cluster_label']],on='area', how='left').drop_duplicates().sort_values(by='area')

##### View the Clustered areas

In [7]:
chd_areas_df

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
3,Chandigarh,Sector 10,30.753276,76.789913,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",5,0
2,Chandigarh,Sector 11,30.758324,76.784144,4.60,4.16,"North Indian,Fast Food,Chinese,Continental,Cafe",5,2
8,Chandigarh,Sector 12,30.766192,76.778375,4.90,4.01,"North Indian,Fast Food,Chinese,Cafe,Continental",1,1
17,Chandigarh,Sector 15,30.751489,76.772605,4.60,4.16,"North Indian,Fast Food,Chinese,Continental,Cafe",9,4
22,Chandigarh,Sector 16,30.746442,76.778375,4.47,3.32,"North Indian,Fast Food,Cafe,Chinese,Italian",1,1
4,Chandigarh,Sector 17,30.739834,76.782702,4.47,3.32,"North Indian,Fast Food,Cafe,Chinese,Italian",2,1
1,Chandigarh,Sector 21,30.726391,76.775490,4.97,5.00,"North Indian,Chinese,Fast Food,Continental,Ita...",5,2
16,Chandigarh,Sector 22,30.733958,76.772620,4.18,3.04,"North Indian,Fast Food,Chinese,Bakery,Street Food",2,1
11,Chandigarh,Sector 23,30.739607,76.766835,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",1,3
0,Chandigarh,Sector 26,30.729959,76.810104,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",6,0


#### Let's Visualize the Neighbouring clusters

In [8]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ------------------------------------------------------------
                         

In [9]:
CITY_LAT,CITY_LNG =  30.7333148,76.7794179
CITY = 'Chandigarh'
map_clusters = folium.Map(location=[CITY_LAT,CITY_LNG],zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat,lon,name,cluster in zip(chd_areas_df['area_latitude'],chd_areas_df['area_longitude'],chd_areas_df['area'],chd_areas_df['cluster_label']):
    # Add Market
    folium.CircleMarker([lat,lon]
                        , radius=5
                        , popup=folium.Popup("{} - {}".format(name,cluster), parse_html=True)
                        , fill=True
                        , color=rainbow[cluster-1]
                        , fill_opacity=0.7).add_to(map_clusters)

# Save the map to HTML file
map_clusters.save(os.path.join(img_path,"areas_clustered_map.html"))

map_clusters

<a id=item2><font size=4 color="229C75">Examine the Clusters</font></a>

Now, you can examine each cluster and determine the discriminating Cuisine categories that distinguish each cluster. Based on the defining categories, let's assign a name to each cluster.

#### Cluster 1

In [10]:
chd_areas_df[chd_areas_df['cluster_label']==0]

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
3,Chandigarh,Sector 10,30.753276,76.789913,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",5,0
0,Chandigarh,Sector 26,30.729959,76.810104,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",6,0
15,Chandigarh,Sector 32,30.709058,76.774397,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",4,0
6,Chandigarh,Sector 37,30.739118,76.749844,3.65,2.49,"North Indian,Fast Food,Chinese,Bakery,Mithai",4,0
10,Chandigarh,Sector 46,30.702621,76.763950,3.77,1.24,"North Indian,Fast Food,Chinese,Street Food,Bev...",6,0


#### Cluster 2

In [33]:
chd_areas_df[chd_areas_df['cluster_label']==1]

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
8,Chandigarh,Sector 12,30.766192,76.778375,4.90,4.01,"North Indian,Fast Food,Chinese,Cafe,Continental",1,1
22,Chandigarh,Sector 16,30.746442,76.778375,4.47,3.32,"North Indian,Fast Food,Cafe,Chinese,Italian",1,1
4,Chandigarh,Sector 17,30.739834,76.782702,4.47,3.32,"North Indian,Fast Food,Cafe,Chinese,Italian",2,1
16,Chandigarh,Sector 22,30.733958,76.772620,4.18,3.04,"North Indian,Fast Food,Chinese,Bakery,Street Food",2,1
19,Chandigarh,Sector 28,30.718075,76.804335,4.57,4.72,"North Indian,Chinese,Continental,Fast Food,Ita...",2,1
5,Chandigarh,Sector 30,30.716292,76.787029,4.81,4.09,"North Indian,Fast Food,Chinese,Cafe,Beverages",1,1
13,Chandigarh,Sector 38,30.745450,76.744480,4.84,4.00,"North Indian,Fast Food,Chinese,Continental,Bev...",3,1
21,Chandigarh,Sector 9,30.746668,76.794240,4.84,4.00,"North Indian,Fast Food,Chinese,Continental,Bev...",2,1


#### Cluster 3

In [34]:
chd_areas_df[chd_areas_df['cluster_label']==2]

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
2,Chandigarh,Sector 11,30.758324,76.784144,4.60,4.16,"North Indian,Fast Food,Chinese,Continental,Cafe",5,2
1,Chandigarh,Sector 21,30.726391,76.775490,4.97,5.00,"North Indian,Chinese,Fast Food,Continental,Ita...",5,2
23,Chandigarh,Sector 34,30.719555,76.763950,4.73,4.84,"North Indian,Chinese,Fast Food,Continental,Ita...",5,2
14,Chandigarh,Sector 47,30.694748,76.769720,4.50,4.68,"North Indian,Chinese,Fast Food,Continental,Bakery",4,2
20,Chandigarh,Sector 7,30.735009,76.804335,4.60,4.50,"North Indian,Fast Food,Chinese,Continental,Bev...",4,2
12,Chandigarh,Sector 8,30.740059,76.798567,4.93,5.00,"North Indian,Chinese,Continental,Fast Food,Ita...",4,2


#### Cluster 4

In [35]:
chd_areas_df[chd_areas_df['cluster_label']==3]

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
11,Chandigarh,Sector 23,30.739607,76.766835,3.77,1.02,"North Indian,Fast Food,Chinese,Bakery,Street Food",1,3
7,Chandigarh,Sector 36,30.732770,76.755294,3.50,0.50,"North Indian,Fast Food,Chinese,Beverages,Stree...",3,3
18,Chandigarh,Sector 40,30.739444,76.737981,4.39,1.99,"North Indian,Fast Food,Chinese,Cafe,Beverages",2,3


#### Cluster 5

In [36]:
chd_areas_df[chd_areas_df['cluster_label']==4]

,city,area,area_latitude,area_longitude,popularity_index,nightlife_index,top_cuisines,restaurants_count,cluster_label
17,Chandigarh,Sector 15,30.751489,76.772605,4.60,4.16,"North Indian,Fast Food,Chinese,Continental,Cafe",9,4
9,Chandigarh,Sector 35,30.726163,76.759622,3.57,2.92,"North Indian,Fast Food,Chinese,Bakery,Street Food",10,4


<a id=item3><font size=4 color="229C75">Conclusion</font></a>

#### After analyzing all the available data and charts we can determine the below points.

<li> If we are planning to open an Expensive Restaurant serving North Indian, Fast food and other Cusines then Sector 15 and Sector 35 are our best options.
<li> If we are planning to open a mid Range Restaurant serving Beverages, Fast food, Chinese or Street Food then Sector 36,23 and 40 are best options due to vicinity with other sectors.
<li> If we are planning to open a low cost, mid range Restaurant serving North India, Continental, Chinese, Cafe we can choose Sector 12,17,22 and nearby vicinity.
<li> If we are planning to open a new bakery then possible choices are Sector 10,26,32,37,46